## **Data Classification**

Gofood merupakan aplikasi delivery food yang populer di Indonesia. Dilansir dari DataIndonesia.id yang bersumber dari laporan Momentum Works, Indonesia tercatat sebagai negara pertama di Asia Tenggara yang paling banyak melakukan layanan pesan-antar makanan di Asia Tenggara. Dimana nilai penjualan bruto Indonesia adalah senilai 4,6 miliar US$ tahun 2023.

Dengan tingginya tingkat penggunaan layanan pesan-antar makanan di Indonesia, memahami kebutuhan, keluhan, dan preferensi pelanggan dari data ini menjadi kunci untuk meningkatkan kualitas layanan, merancang strategi promosi yang lebih efektif, serta mengidentifikasi peluang inovasi produk dan fitur aplikasi.

Oleh karena itu, data ulasan pelanggan GoFood penting untuk dianalisis karena mencerminkan persepsi dan pengalaman langsung pengguna terhadap layanan digital yang semakin berkembang di era ekonomi berbasis platform.

**Install Library**

*Menginstal library pendukung agar dapat menjalankan model AI (seperti klasifikasi dan rangkuman teks) secara langsung dari kode Python melalui layanan pihak ketiga seperti Replicate.*

In [ ]:
!pip install langchain_community
!pip install replicate

In [ ]:
from langchain_community.llms import Replicate
import os
from google.colab import userdata
# Set the API token
api_token = userdata.get('REPLICATE_API_TOKEN')
os.environ["REPLICATE_API_TOKEN"] = api_token
# Model setup
model = "ibm-granite/granite-3.3-8b-instruct"
output = Replicate(
model=model,
replicate_api_token=api_token,
)

**Import Dataset**


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving gofoodreviews.csv to gofoodreviews.csv


In [ ]:
import pandas as pd
df = pd.read_csv("gofoodreviews.csv")
print(df)




                        datetime                   id         username  \
0      2022-06-29 23:59:54+00:00  1542296835255324673          NibobaB   
1      2022-06-29 23:56:41+00:00  1542296024345346048  Agungseptyanto_   
2      2022-06-29 23:55:36+00:00  1542295750897733633        bibime_cc   
3      2022-06-29 23:49:19+00:00  1542294170483556352  skskskskkkskskk   
4      2022-06-29 23:42:29+00:00  1542292450261303299   404Loc_Unknown   
...                          ...                  ...              ...   
28758  2022-06-01 00:19:52+00:00  1531792611161219072   ratiyulyanaptr   
28759  2022-06-01 00:14:19+00:00  1531791213535596544         mbulkola   
28760  2022-06-01 00:11:22+00:00  1531790471298940928     sigdenifikan   
28761  2022-06-01 00:06:39+00:00  1531789284448997377          0xwinow   
28762  2022-06-01 00:06:08+00:00  1531789156338511872        jungjaees   

                                                   tweet  
0      Ada MENU BARU di Ni Boba Brawijaya\n\nWaffle 

In [ ]:
df = pd.read_csv("gofoodreviews.csv")
print(df.columns)

Index(['datetime', 'id', 'username', 'tweet'], dtype='object')


*install library emoji*

In [ ]:
!pip install emoji --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 8.1 MB/s eta 0:00:00


*Menghapus semua karakter emoji, termasuk yang tidak ditangkap oleh regex biasa*

In [ ]:
import emoji
import re

def clean_text(text):
    text = str(text)
    text = re.sub(r"http\S+|www.\S+", "", text)         # Hapus URL
    text = re.sub(r"@\w+", "", text)                    # Hapus mention
    text = re.sub(r"#\w+", "", text)                    # Hapus hashtag
    text = emoji.replace_emoji(text, replace='')        # Hapus emoji dengan library emoji
    text = re.sub(r"\s+", " ", text).strip()            # Hapus spasi berlebih
    return text

df["tweet"] = df["tweet"].apply(clean_text)



*menampilkan seluruh isi kolom tweet 5 teratas*

In [ ]:
pd.set_option('display.max_colwidth', None)
print(df["tweet"].head(5))

0                     Ada MENU BARU di Ni Boba Brawijaya Waffle EGO &amp; BUNGEOPPANG KOREA Link GrabFood : Link ShopeeFood : Link GoFood : NB : * Customer Service
1    Cobain frisco burger. Di daerah pekayon, tp gatau tepatnya di sebelah mana. Soalnya selalu pake gofood/grabfood. Masih jadi yg terbaik di bekasi, menurut saya
2                                                                 Pengin nasi uduk, mager jalan. *buka gofood, gojek, shopeefood *mahal Ujungnya jalan beli sendiri
3                                                          -Aku gofood in ya biar gausa keluar sendirian sama -Aku udah didepan bawa makanan itu beda guys itu beda
4                                                                                  Ih disni banyaaaaaaakk. Tp pas pingin gofood, sajadah depan muka berasa manggil2
Name: tweet, dtype: object


**Provide customer gofood reviews for classification and test the initial prompt**

In [ ]:
sample_reviews = df['tweet'].dropna().head(100).tolist()
reviews_text = "\n".join([f"Review {i+1}: {review}" for i, review in enumerate(sample_reviews)])

prompt = f"""
Klasifikasikan ulasan ini sebagai positif, negatif, atau campuran:

{reviews_text}
"""
# Panggil model dengan contoh prompt
response = output.invoke(prompt)

# Print respon
print("Granite Model Response:\n")

print(response)


Granite Model Response:

1. Positif: Review 1 - This review is positive as it informs about a new menu addition at Ni Boba Brawijaya Waffle EGO & Bungeoppang Korea, available through various food delivery platforms.

2. Positif: Review 2 - This review is positive as the writer expresses satisfaction with the Frisco burger from a restaurant in Pekaynan, Bekasi, accessible via food delivery apps.

3. Mixed: Review 3 - This review is mixed, as the user appreciates the convenience of food delivery apps but finds the prices for nasi uduk too high compared to buying directly.

4. Positif: Review 4 - This review is positive as the user humorously shares their experience with GoFood, implying a generally good experience despite minor inconveniences.

5. Mixed: Review 5 - This review is mixed, as the user expresses enjoyment of GoFood but feels awkward when the food is delivered right in front of them, making them feel like the food is staring at them.

6. Positif: Review 6 - This review is pos

**Refine the prompt to improve the classification output**

In [ ]:
# Menyempurnakan prompt
reviews_text = "\n".join([f"Review {i+1}: {review}" for i, review in enumerate(sample_reviews)])
refined_prompt = f"""
Klasifikasikan ulasan GoFood ini sebagai Positif, Negatif, atau Campuran, dan beri tag pada kategori yang relevan
(kualitas makanan, pengalaman pengguna, pembayaran, waktu pengiriman, atau ongkos kirim)
{reviews_text}
"""
# Panggil model dengan contoh prompt
response = output.invoke(refined_prompt)

# Print respon
print("Granite Model Refined Response:\n")
print(response)

Granite Model Refined Response:

Based on the provided reviews, here's the classification and relevant categories:

1. **Positive** - Review 1, 2, 17, 29, 39, 51, 55, 57, 61, 65, 67, 69, 71, 74, 76, 77, 78, 80, 82, 84, 87, 89, 90, 92, 94, 96, 98, 99

These reviews express satisfaction with GoFood, mention new menus, recommend specific dishes, praise the convenience, and share positive experiences.

2. **Negative** - Review 3, 7, 11, 15, 18, 22, 26, 27, 28, 33, 35, 37, 38, 40, 41, 45, 46, 49, 50, 53, 56, 59, 60, 63, 64, 68, 70, 73, 75, 79, 81, 83, 85, 86, 88, 91, 93, 97

These reviews express dissatisfaction, complain about high prices, driver issues, late deliveries, or technical problems. Some also discuss alternatives to GoFood.

3. **Mixed** - Review 4, 5, 6, 9, 10, 12, 14, 16, 19, 20, 21, 23, 24, 25, 30, 31, 32, 34, 36, 42, 47, 48, 49, 52, 54, 58, 62, 66, 72, 84, 95

These reviews contain a mix of positive and negative sentiments. Some users express appreciation for GoFood's featur

**Added details with 2 steps**

In [ ]:
# Tentukan prompt untuk menyelesaikan tugas dalam 2 langkah
multitask_prompt = f"""
Selesaikan tugas dalam 2 langkah.
Langkah 1: Klasifikasikan ulasan ini sebagai positif, negatif, atau campuran.
Langkah 2: Untuk setiap ulasan, identifikasi kategori yang relevan:
kualitas makanan, pengalaman pengguna, pembayaran, waktu pengiriman, atau ongkos kirim

{reviews_text}
"""
response = output.invoke(multitask_prompt)
print("Granite Model Response:\n")
print(response)

Granite Model Response:

**Langkah 1: Klasifikasikan ulasan ini sebagai positif, negatif, atau campuran.**

1. Positif: Reviews 1, 2, 17, 29, 43, 51, 55, 57, 61, 65, 67, 71, 74, 76, 77, 79, 84, 86, 88, 90, 92, 94, 96, 98, 99. These reviews express satisfaction with the food delivery service, menu variety, ease of use, or recommend the platform.

2. Negatif: Reviews 3, 7, 11, 18, 22, 23, 27, 32, 33, 37, 38, 40, 45, 47, 49, 53, 56, 59, 60, 62, 63, 64, 68, 70, 75, 78, 80, 81, 85, 87, 89, 91, 93, 95, 97, 98. These reviews express dissatisfaction, confusion, or inconvenience with the service, cost, driver behavior, lack of options, technical issues, or promotional problems.

3. Campuran (mixed): Reviews 4, 5, 6, 10, 12, 13, 14, 15, 16, 19, 20, 21, 24, 25, 26, 28, 30, 31, 34, 35, 36, 39, 41, 42, 46, 48, 50, 52, 54, 58, 66, 69, 72, 73, 75, 82, 83, 89, 97. These reviews


In [ ]:
# Tentukan contoh untuk memandu model
formatted_prompt = f"""
Klasifikasikan ulasan ini sebagai Positif, Negatif, atau Campuran, dan beri tag pada kategori yang relevan.
Gunakan format ini: - Sentimen: [Sentimen] - Kategori: [Kategori].
{reviews_text}
"""
# Panggil model dengan prompt
response = output.invoke(formatted_prompt)
# Print respon
print("Granite Model Formatted Response:\n")
print(response)

Granite Model Formatted Response:

- Sentimen: Neutral, Kategori: Food Delivery Service (GoFood)
- Sentimen: Positive, Kategori: Food Quality & Recommendation
- Sentimen: Neutral, Kategori: Food Delivery Service, Cost Comparison
- Sentimen: Neutral, Kategori: Food Delivery Service, Split Bills
- Sentimen: Neutral, Kategori: Account Issue, Food Delivery Service
- Sentimen: Neutral, Kategori: Multi-purpose Purchasing (GoFood, ShopeeFood, PeduliLindungi, MyPertamina)
- Sentimen: Neutral, Kategori: Food Delivery Service, Location Inquiry
- Sentimen: Neutral, Kategori: Food Delivery Service, Personal Preference
- Sentimen: Positive, Kategori: User Experience, Convenience
- Sentimen: Neutral, Kategori: Food Delivery Service, Personal Preference
- Sentimen: Neutral, Kategori: Food Delivery Service, Promotions & Vouchers
- Sentimen: Neutral, Kategori: Food Delivery Service, Location & Reservation
- Sentimen: Positive, Kategori: Food Recommendation, Menu Suggestion
- Sentimen: Neutral, Kategori

In [ ]:
formatted_prompt = f"""
Berikut adalah ulasan pelanggan GoFood.
Klasifikasikan ulasan ini sebagai Positif, Negatif, atau Campuran, dan beri tag pada kategori yang relevan.
Gunakan format ini: - Sentimen: [Sentimen] - Kategori: [Kategori].

Gunakan format ini **untuk setiap review saja** (jangan tulis ulang isi review):

Review 1:
- Sentimen: ...
- Kategori: ...

Review 2:
- Sentimen: ...
- Kategori: ...

{reviews_text}
"""

response = output.invoke(formatted_prompt)
print("Granite Model Refined Response:\n")
print(response)

Granite Model Refined Response:

Review 1: 
- Sentimen: Neutral
- Kategori: Menu, Promotion

Review 2:
- Sentimen: Positive
- Kategori: Food Quality, Location

Review 3:
- Sentimen: Negative
- Kategori: Pricing, Convenience

Review 4:
- Sentimen: Neutral
- Kategori: Order Process

Review 5:
- Sentimen: Positive
- Kategori: Food Presentation

Review 6:
- Sentimen: Positive
- Kategori: Payment, Split Bill

Review 7:
- Sentimen: Negative
- Kategori: Account Issue, Customer Support

Review 8:
- Sentimen: Neutral
- Kategori: Shopping, Food Delivery

Review 9:
- Sentimen: Inquiry
- Kategori: Delivery Cost, Intercity Delivery

Review 10:
- Sentimen: Neutral
- Kategori: Promotion, Comparative Analysis

Review 11:
- Sentimen: Negative
- Kategori: Account Issue, Customer Support

Review 12:
- Sentimen: Humorous
- Kategori: User Engagement, Random

Review 13:
- Sentimen: Positive
- Kategori: Promotion, Discount

Review 14:
- Sentimen: Humorous
- Kategori: Food Craving, Pop Culture Reference

Revi

**SUMMARIZING**

Berdasarkan data classification di atas, maka langkah selanjutnya untuk mempermudah membaca data ulasan gofood maka diperlukan ringkasan dengan bahasa yang mudah dipahami juga. Disini diambil 100 ulasan untuk membuat ringkasan.

In [ ]:

# Mengambil 100 ulasan
sample_reviews = df['tweet'].dropna().head(100).tolist()

# Menggabungkan ulasan
reviews_text = "\n".join([f"- {review}" for review in sample_reviews])

# Membuat prompt ringkasan
summary_prompt = f"""
Buat ringkasan naratif dalam bahasa Indoensia dari kumpulan ulasan pelanggan GoFood berikut ini.
Soroti kesan umum, keluhan yang sering muncul, dan hal-hal yang disukai pelanggan:

{reviews_text}
"""

# Kirim ke model
response = output.invoke(summary_prompt)

# Print respon
print("Summary Dataset:\n")
print(response)


Summary Dataset:

**Ringkasan Naratif Pelanggan GoFood:**

**Kesan Umum dan Keluhan Sering:**

1. **Menu Baru dan Promosi:** Pelanggan menyukai kedatangan menu baru, seperti Ni Boba Brawijaya Waffle EGO & Bungeoppang Korea, dan sering mencari promosi atau diskon.

2. **Kelemahan Jangka Panjang:** Beberapa pelanggan mengatakan bahwa GoFood lebih mahal dibandingkan dengan membeli sendiri, khususnya untuk makanan jangka panjang.

3. **Teksasnya Pembayaran:** Ada keluhan tentang kesulitan menggunakan voucher atau kode promo, serta masalah dengan ongkir yang dipertimbangkan.

4. **Kecurangan Akun:** Terdapat beberapa kasus pelanggan yang mengalami "soft ban" atau kesulitan menggunakan akun mereka, yang menyebabkan kekecewaan dan permintaan bantuan.

5. **Kualitas Makanan:** Ada komentar positif tentang kualitas makanan dari beberapa restoran tertentu, seperti KFC, Dapur Gudeg Bu Tutie, dan Flash Coffee.

6. **Kelemahan Jam Lebih Lalu:** Ada keluhan tentang kesulitan mendapatkan GoFood pada 

In [ ]:
refined_prompt = f"""
Buat ringkasan naratif dalam bahasa Indonesia dari kumpulan ulasan pelanggan GoFood berikut ini.
Tulis dalam bentuk paragraf (bukan bullet point).

{reviews_text}
"""

response = output.invoke(refined_prompt)
print(response)

Ringkasan Naratif:

Kumpulan ulasan pelanggan GoFood menyampaikan variasi pengalaman mereka saat menggunakan layanan ini. Beberapa ulasan menyebutkan ketersediaan menu baru di restoran tertentu, seperti Ni Boba Brawijaya Waffle dan bungeoppang Korea. Ulasan lainnya membahas pengalaman mereka saat mencari makanan khusus di daerah tertentu, seperti Frisco burger di Pekayanan atau nasi uduk yang mahal jika dibeli secara langsung.

Selain itu, ada ulasan yang menunjukkan kemudahan dalam menggunakan GoFood untuk belanja bersama, seperti fitur split bills, dan pilihan pembayaran yang bervariasi, termasuk GoPay, kode promo, dan kemudahan penyetoran saldo ke aplikasi lain. Namun, ada juga komplain tentang akun yang terblokir tanpa alasan jelas dan permintaan bantuan dari pihak GoFood.

Beberapa pengguna menyampaikan kesenian mereka terhadap makanan yang diorder melalui GoFood, dari minuman sederhana hingga makanan istimewa seperti ayam geprek, sushi tei, dan hidangan regional seperti krecek da

In [ ]:

# Buat prompt untuk summarization
summary_prompt = f"""
Berikut adalah 100 ulasan pelanggan GoFood.
Buat ringkasan naratif dalam bahasa Indonesia yang menyampaikan:
1. Kesan umum pelanggan terhadap layanan GoFood
2. Keluhan yang sering muncul
3. Hal-hal yang disukai pelanggan

Ulasan:
{reviews_text_summary}
"""

# Kirim ke model
summary_response = output.invoke(summary_prompt)

# Tampilkan hasil ringkasan
print("\n=== Ringkasan 100 Ulasan dalam Bahasa Indonesia ===\n")
print(summary_response)



=== Ringkasan 100 Ulasan dalam Bahasa Indonesia ===

**Ringkasan Naratif Ulasan Pelanggan GoFood:**

1. **Kesan Umum Pelanggan Terhadap Layanan GoFood:**
   - Pelanggan senang dengan kemudahan layanan GoFood dalam membeli makanan dari restoran berbeda, terutama ketika mereka tidak ingin keluar sendirian.
   - Ada kesulitan dalam mendapatkan promosi atau diskon yang berkembang.
   - Beberapa pelanggan mengalami masalah dengan akun mereka, terutama dalam mendapatkan voucher atau kode promo.

2. **Keluhan yang Sering Muncul:**
   - Masalah dengan ongkir yang mahal, terutama untuk makanan yang diorder ke jauh.
   - Tidak ada fitur split bills secara umum di beberapa restoran, menyebabkan pelanggan harus bayar secara terpisah.
   - Ada keluhan tentang kualitas makanan yang diterima, terutama ketika makanan tidak seperti yang diharapkan.
   - Pelanggan mengalami kesulitan dalam mendapatkan jaminan pelayanan yang baik, terutama pada waktu malam.

3. **Hal-hal yang Disukai Pelanggan:**
   - P

In [ ]:
summary_prompt = f"""
Buatlah ringkasan dalam bahasa Indonesia dari ulasan-ulasan pelanggan GoFood dan
tuliskan hasilnya dalam 3 bagian, masing-masing **berupa bullet poin**:
1. Kesan umum pelanggan terhadap layanan GoFood
2. Keluhan yang sering muncul
3. Hal-hal yang disukai pelanggan

Tulis dengan rapi dalam format:
**Kesan umum pelanggan terhadap layanan GoFood**
**Keluhan yang sering muncul**
**Hal-hal yang disukai pelanggan**


{reviews_text}
"""

response = output.invoke(summary_prompt)
print(response)

**Kesan umum pelanggan terhadap layanan GoFood**

- Pelanggan menyukai kemudahan dan kesesuaian menu yang bervariasi.
- Beberapa pelanggan mengungkapkan strategi untuk menjimatkan biaya makan dengan menggunakan promo dan kode voucher.
- Ada keluhan tentang keterbatasan waktu pembayaran BPJS melalui aplikasi GoFood.

**Keluhan yang sering muncul**

- **Keterbatasan menu dan harga yang mahal**: Pelanggan mengatakan bahwa beberapa restoran yang terdaftar di GoFood memiliki harga yang lebih tinggi dibandingkan pemasaran mereka.
- **Masalah pembayaran dan kode promo**: Pelanggan mengalami kesulitan dalam menggunakan kode promo dan voucher, dan ada yang ditangkap oleh sistem soft-banned.
- **Kualitas layanan dan driver**: Ada keluhan tentang kualitas layanan yang tidak consistent, serta driver yang tidak mau bekerja pada waktu tertentu.
- **Ongkir yang tidak sehati**: Ada keluhan tentang ongkir yang dikenakan tidak sesuai dengan jangka waktu dan jarak yang diambil.

**Hal-hal yang disukai pe

In [ ]:
summary_prompt = f"""
Berikut ini adalah kumpulan ringkasan insight dari ulasan pelanggan GoFood:

{reviews_text}

Berdasarkan ringkasan tersebut, buatlah evaluasi dengan format sebagai berikut:

- **Kategori**: [tuliskan tema utama atau masalah yang diangkat]
- **Evaluasi & Rekomendasi**: [berikan evaluasi singkat dan rekomendasi konkret untuk perbaikan atau penguatan layanan]

Berikan 2 poin evaluasi yang mencakup kekuatan dan kelemahan layanan GoFood.
Tulis dalam bahasa Indonesia yang jelas dan profesional.
"""

response = output.invoke(summary_prompt)
print(response)

# Menyimpan hasil ke file agar tidak hilang atau terpotong
with open("evaluasi_gofeed.txt", "w", encoding="utf-8") as f:
    f.write(response)


### Evaluasi & Rekomendasi

#### Kategori 1: Menu & Pilihan Makanan
**Kekuatan:**
- GoFood menawarkan berbagai pilihan menu dari berbagai jenis makanan, termasuk makanan tradisional hingga makanan internasional.
- Ada pilihan khusus untuk acara tertentu, seperti pesan untuk nonton MV BTS atau makan krecek dan gudeg di Dapur Gudeg Bu Tutie.

**Kelemahan:**
- Beberapa pelanggan menyebutkan kesulitan mendapatkan promosi atau voucher yang valid, yang dapat menurunkan kepuasan pelanggan.
- Keterbatasan menu di beberapa wilayah, seperti di Padang malam jam 10, ketika semua restoran tersebut tidak menerima order.

**Rekomendasi:**
- GoFood harus memperhatikan dan meningkatkan sistem promosi dan voucher agar lebih terjangkau dan berlaku dengan lancar.
- Pengelolaan menu harus diperhatikan untuk memastikan ada pilihan makanan di setiap wilayah dan waktu.

#### Kategori 2: Kualitas Layanan & Pembayaran
**Kekuatan:**
- Pelayanan cepat dan efisien, dengan waktu pengambilan makanan relatif singkat.

In [ ]:
# Menyimpan hasil evaluasi dan rekomendasi ke file txt
with open("evaluasi_gofod_final.txt", "w", encoding='utf-8') as file:
    file.write(response)  # response = hasil dari output.invoke()

from google.colab import files
files.download("evaluasi_gofod_final.txt")



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

*Conclusion*

Berdasarkan hasil analisis ulasan pelanggan, dapat disimpulkan bahwa layanan GoFood secara umum mendapat tanggapan positif, terutama dalam hal variasi menu dan program promosi. Namun, masih terdapat sejumlah tantangan yang perlu menjadi perhatian, seperti perbaikan sistem pembayaran, peningkatan aksesibilitas restoran, serta penyempurnaan fitur-fitur layanan untuk menghindari pengalaman negatif. Sebagai data analyst, temuan ini dapat menjadi dasar rekomendasi strategis bagi manajemen dalam mengambil keputusan berbasis data untuk meningkatkan kualitas layanan dan kepuasan pelanggan secara berkelanjutan.